In [25]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/src/utils")
from revert_normalisation import get_ct_normalisation_values, revert_normalisation


In [22]:
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

data_root = "/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1"
list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
print("input1 ---", len(list_data_mri), list_data_mri)
print("input2 ---", len(list_data_mask), list_data_mask)

input1 --- 89 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA002/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA008/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA010/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA017/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA021/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA023/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA032/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val_Input/Task1/AB/1ABA037/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synth

In [23]:
# copy files to nnUNet raw data directory


dataset_id = 200
dataset_data_name = "synthrad2025_task1_MR"
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 

def process_file(data_path, dataset_path, modality_suffix="_0000"):
    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'{modality_suffix}.mha'):
        filename = filename + f'{modality_suffix}.mha'
    os.makedirs(os.path.join(dataset_path, 'imagesVal'), exist_ok=True)
    shutil.copyfile(data_path, os.path.join(dataset_path, f'imagesVal/{filename}'))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))


100%|██████████| 89/89 [00:02<00:00, 35.04it/s]


In [24]:
input_dir = os.path.join(dataset_data_path, 'imagesVal')
submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
output_dir = os.path.join(submission_path, f'Dataset{dataset_id:03d}_{dataset_data_name}_submission')
fold = 0

In [20]:
input_dir

'/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset202_synthrad2025_task1_MR_mask/imagesVal'

In [19]:
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p nnUNetPlans -tr nnUNetTrainerMRCT -f {fold}]")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



Traceback (most recent call last):
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/envs/nnunet_trans/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/inference/predict_from_raw_data.py", line 896, in predict_entry_point
    args.f = [i if i == 'all' else int(i) for i in args.f]
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/inference/predict_from_raw_data.py", line 896, in <listcomp>
    args.f = [i if i == 'all' else int(i) for i in args.f]
ValueError: invalid literal for int() with base 10: '0]'


256

In [ ]:
nnUNetv2_predict -d 202 -i /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset202_synthrad2025_task1_MR_mask/imagesVal -o /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission -c 3d_fullres -p nnUNetPlans -tr nnUNetTrainerMRCT -f 0 -chk checkpoint_best.pth

In [26]:

ct_plan_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset203_synthrad2025_task1_CT/nnUNetPlans.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
pred_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/Dataset202_val_pred"
revert_normalisation(pred_path, ct_mean, ct_std, save_path=pred_path + "_revert_norm")

CT mean: -726.3590698242188, CT std: 444.69049072265625


100%|██████████| 89/89 [01:56<00:00,  1.30s/it]


In [ ]:
save_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/Folder"
os.makedirs(save_path, exist_ok=True)
for file in glob.glob(os.path.join(pred_path + "_revert_norm", "*.mha")):
    new_file = os.path.join(save_path, "sct_" + os.path.basename(file))
    shutil.copyfile(file, new_file)
# zip folder
shutil.make_archive(save_path, 'zip', save_path)


'/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/Folder.zip'

In [ ]:


#rsync -avz xin015@virga.hpc.csiro.au:/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/Folder.zip ./test
# 